In [1]:
%pip install pandas aiohttp plotly.express asyncio nbformat

Note: you may need to restart the kernel to use updated packages.


In [2]:
import aiohttp
import asyncio
import json
import plotly.express as px
import pandas as pd
from itertools import product
from functools import reduce

In [3]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            keyJson = await resp.json()

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            tokenJson = await resp.json()
    return tokenJson.get("token", None)

In [4]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

In [5]:
def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

In [6]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

In [7]:
queryStr = """
{
surveyPage{
    id
    name
    # questions{
    #     id
    #     name
    #     order
    #     type{
    #         id
    #         name
    #     }
    #     answers{
    #         id
    #         value
    #         answered
    #         user{
    #             id
    #             email
    #         }
    #     }
    # }
    }
}

"""

In [8]:
mappers = {
     "id": "id",
    "name": "name",
    # "questions": "questions",
    # "questionsid": "questions.id",
    # "questionsname": "questions.name",
    # "questionsorder": "questions.order",
    # "questionsTypeid": "questions.type.id",
    # "questionsTypename": "questions.type.name",
    # "questionsAnswersid": "questions.answers.id",
    # "questionsAnswersvalue": "questions.answers.value",
    # "questionsAnswersanswered": "questions.answers.answered",
    # "questionsAnswerUserid": "questions.answers.user.id",
    # "questionsAnswerUseremail": "questions.answers.user.email"
}

In [9]:
async def fullPipe():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("surveyPage", None)
    
    resultMapped = list(map(lambda project: {**project, "surveyCount": len(project.get("surveys", []))}, result))
    flatData = flatten(resultMapped, {}, mappers)
    return list(flatData)

async def main():
    flatData = await fullPipe()
    with open('resultNotebook.json', "w", encoding='utf-8') as outputFile:
        json.dump(flatData, outputFile)
    print(flatData)

await main()

[{'id': '910d54a9-7f2e-41ca-b811-3c600ef82fda', 'name': 'Studentské hodnocení 2022'}]
